https://www.bilibili.com/video/BV1MB4y1F7of?p=2

# 多输入多输出通道

实现一下多输入输出通道互相关运算

##  多输入通道的互相关运算

In [1]:
import torch
from d2l_torch import d2l

# 如果是mac要重新规定一下线程数量
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [6]:
def corr2d_multi_in(X,K):
    """
    多输入二维卷积的计算实现
    X 表示3d的输入（每次拿出一个通道的矩阵来）
    K 表示3d的卷积核(每次拿出一个卷积核来)
    """
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

![img](https://img-blog.csdnimg.cn/cfc3c3143919489f9e4ab7de7939175b.jpg)

In [7]:
# 比如，以上图中的输入和核来验证一下我们刚刚写的多输入通道函数的计算
X=torch.tensor([
                [[0.,1.,2.],[3.,4.,5.],[6.,7.,8.]],
                [[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]]
               ])
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道的互相关运算

In [31]:
def corr2d_multi_out(X,K):
    """
    计算多输出通道的互相关运算
    X表示输入，是一个3d的张量
    K表示卷积核，多输出通道的卷积核是一个4d张量，(c_o,c_i,k_h,k_w) 
    """
    return torch.stack([corr2d_multi_in(X,k) for k in K],dim=0)

In [32]:
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])
K=torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [33]:
corr2d_multi_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

+ 多输入通道中，包含两个$2\times 2$的卷积核，输出是$2\times 2$
+ 多输出通道中，输出通道为3，包含3组卷积核，其中每组卷积核都包含两个$2\times 2$的卷积核，输出是$3\times 2\times 2$

## 1X1的卷积等价于全连接

In [35]:
def corr2d_multi_out_1x1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    X=X.reshape(c_i,h*w)
    # 转换成全连接，c_i个输入，每个输入是h*w维度
    K=K.reshape((c_o,c_i))
    # 因为后面两维其实是(1,1)
    Y=torch.matmul(K,X)
    # 这里没有进行加偏置
    return Y.reshape((c_o,h,w))

X=torch.normal(0,1,(3,3,3))
# 输入是三通道，每个通道是(3,3)
K=torch.normal(0,1,(2,3,1,1))
# 卷积核是两组，每组有3个(1,1)的卷积核

Y1=corr2d_multi_out_1x1(X,K)
Y2=corr2d_multi_out(X,K)

print(f"Y1={Y1}, \n Y2={Y2}")

assert float(torch.abs(Y1-Y2).sum())<1e-6

Y1=tensor([[[ 0.8397,  0.4864,  0.3800],
         [ 0.1702,  0.0867,  0.1117],
         [ 0.1599, -0.2185, -0.4287]],

        [[-0.2285, -1.5198, -4.3252],
         [-5.8408, -0.7174,  1.3526],
         [ 1.2860, -0.2413, -0.9581]]]), 
 Y2=tensor([[[ 0.8397,  0.4864,  0.3800],
         [ 0.1702,  0.0867,  0.1117],
         [ 0.1599, -0.2185, -0.4287]],

        [[-0.2285, -1.5198, -4.3252],
         [-5.8408, -0.7174,  1.3526],
         [ 1.2860, -0.2413, -0.9581]]])


##  torch的和numpy的stack，cat（concat)

In [17]:
import numpy as np

In [30]:
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])
a=torch.stack((K,K+1,K+2),0)
K.shape,a.shape,a
# 所以K，K+1，K+2stack构成a时，只需要把这三个元素放到一起，然后最外层套一层/一个轴就可以

(torch.Size([2, 2, 2]),
 torch.Size([3, 2, 2, 2]),
 tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]))

In [26]:
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])

a=np.stack((K,K-1,K-2),-1)

K,K-1,K-2,a,a.shape
# 如果是-1，则最里面那层是新加的轴。
# 从结果不难发现，其实就是K，K-1，K-2中每个位置的元素，取一个。
# 比如，stack后的结果最内层的第一个元素是[0,-1,-2]。其实就分别对应于K，K-1，K-2中的[0,0,0]号元素。

(tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]),
 tensor([[[-1.,  0.],
          [ 1.,  2.]],
 
         [[ 0.,  1.],
          [ 2.,  3.]]]),
 tensor([[[-2., -1.],
          [ 0.,  1.]],
 
         [[-1.,  0.],
          [ 1.,  2.]]]),
 array([[[[ 0., -1., -2.],
          [ 1.,  0., -1.]],
 
         [[ 2.,  1.,  0.],
          [ 3.,  2.,  1.]]],
 
 
        [[[ 1.,  0., -1.],
          [ 2.,  1.,  0.]],
 
         [[ 3.,  2.,  1.],
          [ 4.,  3.,  2.]]]], dtype=float32),
 (2, 2, 2, 3))

In [21]:
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])
K=np.stack((K,K+1,K+2),0)
K.shape

(3, 2, 2, 2)

看上面的代码，其实
```python
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])

torch.stack()
Concatenates a sequence of tensors along a new dimension.
All tensors need to be of the same size.


numpy.stack()
Join a sequence of arrays along a new axis.

The ``axis`` parameter specifies the index of the new axis in the
dimensions of the result. For example, if ``axis=0`` it will be the first
dimension and if ``axis=-1`` it will be the last dimension.

```

+ 所以这里对三个K进行stack，这里torch的stack和numpy的stack函数其实一模一样，如果轴dim=0，就在最前面加一个轴，如果=-1，就在最后面加一个轴
+ 容易搞混的是，np.hstack(),np.vstack()这两个一个是水平方向拼接，一个是竖直方向拼接，常见于二维数据的操作，比如dataframe。torch也有这几个函数
+ 所以torch和numpy的函数基本一模一样，区别就是一个在cuda上实现过！另一个没有

---

另外，torch中对应于stack，其实也有concat，但是一般常写作为cat,如下示例
+ concat不同于stack，不会增加新的轴，只会在某个指定的轴上对元素进行拼接
+ torch中的cat功能和numpy中的concat一模一样

In [13]:
K=torch.tensor([[[0.,1.],[2.,3.]],
                [[1.,2.],[3.,4.]]
               ])
a= torch.cat((K,K+1,K+2),0)
a.shape

torch.Size([6, 2, 2])

In [14]:
a= torch.cat((K,K+1,K+2),1)
a.shape

torch.Size([2, 6, 2])

In [16]:
a= torch.cat((K,K+1,K+2),2)
a.shape

torch.Size([2, 2, 6])